In [27]:
import functools
import itertools
import math
import re
import statistics
from functools import reduce
from pprint import pprint
from typing import (
    List,
    Optional,
    Generator,
    NamedTuple,
    Tuple,
    Set,
    Any,
    Dict,
    Union,
    Callable,
    Iterable,
    Sequence,
)

import rich.panel
import sklearn.base
from alive_progress import alive_bar
from numpy.distutils.misc_util import is_sequence
from tqdm import tqdm, trange
from log_symbols import LogSymbols

import overrides as overrides
import pandas as pds
import tabulate as tabulate
from collections import namedtuple
from sklearn import linear_model, metrics, preprocessing, model_selection, svm
from tabulate import tabulate
from pathlib import Path
import pickle
from halo import Halo
import random
import argparse
import sys
from sklearn.experimental import enable_halving_search_cv
from sklearn import neural_network, tree
from sklearn.model_selection import (
    HalvingGridSearchCV,
    RandomizedSearchCV,
    HalvingRandomSearchCV,
)
import scipy.stats as scistats

import rich.progress
import commons
from data_types import PickleOut
from rich.traceback import install as niceTracebacks
from rich.table import Table as rTable

from db_actions import db_actions
import numpy as np

from linRegPred import getSplits, getTransformers


In [28]:
with db_actions.connect() as conn:
    dF = pds.read_sql('SELECT * FROM "runtimeScorePredBase1000"', conn)
# print(dF)
all_x_cols = [
        "build-linux-kernel1",
        "fio2",
        "fio3",
        "fio4",
        "fio5",
        "fio6",
        "fio7",
        "fio8",
        "fio9",
        "iperf10",
        "iperf11",
        "iperf12",
        "iperf13",
        "john-the-ripper14",
        "john-the-ripper15",
        "ramspeed16",
        "ramspeed17",
        "ramspeed18",
        "ramspeed19",
        "ramspeed20",
        "ramspeed21",
        "ramspeed22",
        "ramspeed23",
        "ramspeed24",
        "ramspeed25",
        "stream26",
        "stream27",
        "stream28",
        "stream29",
        "pCpu",
        "cpus",
        "rss",
        "vmem",
        "rchar",
        "wchar",
        "syscr",
        "syscw",
]
filtered_x_cols = ['john-the-ripper15', 'ramspeed17', 'ramspeed18', 'ramspeed19', 'ramspeed20', 'ramspeed21', 'ramspeed23', 'ramspeed24', 'ramspeed25', 'pCpu', 'cpus', 'vmem', 'rchar', 'wchar', 'syscr', 'syscw']
x_cols = filtered_x_cols
y_cols = "realtime"

X: pds.DataFrame = dF[x_cols]
y: pds.DataFrame = dF[y_cols]

In [ ]:
t = rTable(title="results")
t.add_column("cv")
t.add_column("unknown")
t.add_column("cv score")
t.add_column("unknown score")
for split in getSplits(dF, 1, x_cols, y_cols, cvSize=1, unknownSize=1):
    X_train, y_train, X_test, y_test, X_unknown, y_unknown, X_trans, ukwfs, cvwfs = split
    model = tree.DecisionTreeRegressor()
    model.fit(X_train, y_train)
    # print(cvwfs, ukwfs, model.score(X_test, y_test), model.score(X_unknown, y_unknown))
    t.add_row(cvwfs[0], ukwfs[0], str(model.score(X_test, y_test)), str(model.score(X_unknown, y_unknown)))
rich.print(t)

In [ ]:
X_trans = getTransformers(X, 1)
scale1, poly, scale2 = X_trans
X_scaled = scale2.transform(poly.transform(scale1.transform(X)))
res = list()
for i in range(100):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X_scaled, y, test_size=0.2)
    model = tree.DecisionTreeRegressor()
    model.fit(X_train, y_train)
    res.append(model.score(X_test, y_test))

In [ ]:
rich.print(np.average(res), np.std(res))

In [29]:
taskNames = pds.unique(dF.taskName)


def pickCVTasks():
    cvTasks = list(taskNames)
    totalData = len(dF)
    while len(dF.query("taskName in @cvTasks")) > .3 * totalData:
        cvTasks = list()
        remainingTasks = list(taskNames)
        while len(dF.query("taskName in @cvTasks")) < .2 * totalData:
            pick = np.random.choice(remainingTasks)
            remainingTasks.remove(pick)
            cvTasks.append(pick)
    return cvTasks


nodeIds = pds.unique(dF.nodeConfig)


def pickCVNodes():
    cvNodes = list(nodeIds)
    totalData = len(dF)
    while len(dF.query("nodeConfig in @cvNodes")) > .85 * totalData:
        cvNodes = list()
        remainingNodes = list(nodeIds)
        while len(dF.query("nodeConfig in @cvNodes")) < .8 * totalData:
            pick = np.random.choice(remainingNodes)
            remainingNodes.remove(pick)
            cvNodes.append(pick)
    return cvNodes

In [30]:
def listProd(l):
    return reduce(lambda a, x: a * x, l, 1)

In [31]:
X_trans = getTransformers(X, 1)
scale1, poly, scale2 = X_trans
X_scaled = scale2.transform(poly.transform(scale1.transform(X)))
res = list()
done = set()
for i in range(100):
    cvNodes = pickCVNodes()
    #while listProd([i - 164 for i in cvNodes]) in done:
    #    cvNodes = pickCVNodes()
    #done.add(listProd([i - 164 for i in cvNodes]))
    train = dF.query("nodeConfig not in @cvNodes")
    X_train = train[x_cols]
    X_train = scale2.transform(poly.transform(scale1.transform(X_train)))
    y_train = train[y_cols]
    test = dF.query("nodeConfig in @cvNodes")
    X_test = test[x_cols]
    X_test = scale2.transform(poly.transform(scale1.transform(X_test)))
    y_test = test[y_cols]
    model = tree.DecisionTreeRegressor()
    #model = linear_model.LinearRegression()
    model.fit(X_train, y_train)
    res.append((cvNodes, model.score(X_test, y_test)))

In [32]:
t = rTable(title="results")
t.add_column("cv tasks")
t.add_column("score")
for r in res[:5]:
    cv, s = r
    print(cv, s)
    #t.add_row(", ".join(list(cv)), str(s))
print("average:", np.average([s for cv, s in res]), "std:", np.std([s for cv, s in res]))
#rich.print(t)


[167, 178, 184, 189, 192, 182, 169, 170, 179, 187, 191, 166, 181, 165, 168, 171, 172, 175, 188, 180, 193, 186] 0.9118759636342564
[187, 183, 182, 165, 175, 172, 184, 180, 173, 188, 192, 179, 186, 178, 191, 167, 190, 166, 169, 176, 181, 185] 0.8772595314415776
[172, 186, 175, 190, 192, 188, 181, 180, 187, 171, 169, 179, 183, 191, 173, 184, 166, 167, 182, 178, 165, 170] 0.8955091650613322
[185, 181, 166, 180, 190, 179, 168, 192, 175, 167, 173, 193, 170, 178, 187, 186, 171, 188, 172, 191, 169, 184] 0.9209683087890367
[175, 166, 172, 187, 190, 191, 176, 184, 181, 183, 193, 169, 192, 173, 178, 168, 188, 186, 179, 171, 167, 185] 0.9214343784076142
average: 0.8845131692779443 std: 0.04664187183458884


In [21]:
tmp = dict()
seen = dict()
for r in res:
    cv, s = r
    for c in cv:
        if c not in tmp.keys():
            tmp[c] = 0
            seen[c] = 0
        tmp[c] = tmp[c] + s / len(cv)
        seen[c] = seen[c] + 1
for k in tmp.keys():
    tmp[k] = tmp[k] / seen[k]
t1 = tmp.keys()
t2 = tmp.values()
t = list(zip(t1, t2))
print(sorted(t, key=lambda x: x[1]))
print(np.average(list(t2)), np.std(list(t2)))

[(170, 0.04028053635860174), (183, 0.04029801920233973), (180, 0.040298742156475206), (168, 0.040319782367888926), (192, 0.04033043355258681), (181, 0.04035885691424642), (186, 0.040362572584791256), (185, 0.04036583177935518), (182, 0.040369344857983505), (193, 0.04037284791298116), (165, 0.04037545257172535), (187, 0.04039085935410132), (179, 0.04039099429330804), (172, 0.04039603013279658), (166, 0.0403988508756666), (167, 0.04039903367842179), (173, 0.04040689074400878), (190, 0.04041256492346465), (175, 0.04041589013152856), (169, 0.04042175402727249), (178, 0.04042445914477014), (176, 0.040449038370745975), (191, 0.04045089257760816), (188, 0.04047747528241734), (184, 0.04058861225684818), (189, 0.040601681521309364), (171, 0.04066593687852045)]
0.04040827349821347 8.831200077538215e-05
